In [ ]:
!cp -r ../input/sentence-transformers/* . && pip install -e .

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

test_df = pd.read_csv(f'../input/us-patent-phrase-to-phrase-matching//test.csv')
titles = pd.read_csv(f'../input/cpc-codes//titles.csv')

test_df = test_df.merge(titles, left_on='context', right_on='code')
test_df['input'] = test_df['title'] + ' ' + test_df['anchor']

model = SentenceTransformer('../input/all-mpnet-base-v2/')

predictions = []
for i in test_df.iterrows():
    emb1 = model.encode(i[1]['input'])
    emb2 = model.encode(i[1]['target'])

    cos_sim = util.cos_sim(emb1, emb2)
    predictions.append(cos_sim.item())
    
test_df['score'] = predictions
test_df[['id', 'score']].to_csv('submission.csv', index=False)